In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


In [ ]:
# Load datasets
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')


In [ ]:
# Add a time-related feature, 'age' of the vehicle
current_year = 2024  # Assuming the current year is 2024
train_data['age'] = current_year - train_data['model_year']
test_data['age'] = current_year - test_data['model_year']

In [ ]:
# Features and target variable
X_train = train_data.drop(columns=['price', 'id'])
y_train = train_data['price']
X_test = test_data.drop(columns=['id'])

In [ ]:
# Function to extract numeric horsepower
def extract_hp(engine_str):
    match = re.search(r'(\d+\.\d+|\d+)HP', engine_str)
    if match:
        return float(match.group(1))
    else:
        return np.nan

# Apply function to extract horsepower
if 'engine' in X_train.columns:
    X_train['engine_hp'] = X_train['engine'].apply(extract_hp)
if 'engine' in X_test.columns:
    X_test['engine_hp'] = X_test['engine'].apply(extract_hp)

In [ ]:
# Drop the original 'engine' feature
X_train.drop(columns=['engine'], inplace=True, errors='ignore')
X_test.drop(columns=['engine'], inplace=True, errors='ignore')

# Drop rows with NaN values in 'engine_hp' (if any)
X_train = X_train.dropna(subset=['engine_hp'])
y_train = y_train.loc[X_train.index]

In [ ]:
# Define numerical and categorical features
numeric_features = ['model_year', 'milage', 'engine_hp', 'age']
categorical_features = ['brand', 'model', 'fuel_type', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']

# Create transformers for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine transformers into a single preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocess training and testing data
X_train_processed = preprocessor.fit_transform(X_train).toarray()  # Convert to dense array
X_test_processed = preprocessor.transform(X_test).toarray()        # Convert to dense array

# Train the SVR model
svr = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
svr.fit(X_train_processed, y_train)

# Predict on the test set
y_test_pred = svr.predict(X_test_processed)

In [ ]:
# Ensure all IDs have predictions by creating a DataFrame with all IDs
submission = pd.DataFrame({
    'id': test_data['id'],
    'price': y_test_pred.flatten()
})

# Save the predictions to a CSV file in the format [id, price]
submission.to_csv('/content/SVM_prices.csv', index=False)

In [ ]:
# Since we don't have true price labels for the test set, we'll calculate RMSE on a portion of the training data held out as validation set
# Split the training data into a smaller training set and a validation set
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train the best model on the split training data
best_model.fit(X_train_split, y_train_split)

# Predict on the validation set
y_val_pred = best_model.predict(X_val)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"Validation RMSE: {rmse:.2f}")

# Save the predictions to a CSV file in the format [id, price]
submission = pd.DataFrame({
    'id': test_data['id'],
    'price': y_test_pred
})
submission.to_csv('/content/predicted_prices_Elasticnet.csv', index=False)

Validation RMSE: 66195.88
